# Dynamics experiments

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import matplotlib.patches as pltp
from matplotlib.animation import FuncAnimation, PillowWriter
from numpy import exp, pi, array, asarray
from sympy.solvers import solve
from sympy import Symbol, Ellipse, Point, Line, Matrix, diff


## Analytical kinematics

In [ ]:
def get_act(p1, p2, act_len):
    '''Get coordinates of the actuator based on two joints' positions'''
    dist = sp.sqrt((p1.x - p2.x) ** 2 + (p1.y - p2.y) ** 2)
    new_len = dist + act_len
    p12 = Point(new_len / dist * (p2 - p1) + p1)
    return p12

In [ ]:
t_begin = 0
steps = 100
t_end = np.abs(4 * np.pi / w)
dt = (t_end - t_begin) / steps
ts = np.linspace(t_begin, t_end, steps)
t = sp.Symbol('t')

delta12 = (12, 0,)               # Distance between two mechanisms

w1 = 1
a1 = 5*np.pi/6                                  
l1 = .1                                      
R1, R2 = 1, 1.5
center1 = Point(0, 0)                            
center2 = Point(
    center1[0] + (R1 + R2 + l1)*np.cos(a1), 
    center1[1] + (R1 + R2 + l1)*np.sin(a1)
)
d1, d2 = 0.8, 1.4                                  
phi11, phi12 = 0, np.pi                              
act_len1 = 2


In [ ]:
# Position of revolute joints and end effectors
rev1 = Point(center1.x + d1 * sp.cos(w1 * t + phi11), center1.y + d1 * sp.sin(w1 * t + phi11))
rev2 = Point(center2.x + d2 * sp.cos(w1 * t + phi12), center2.y + d2 * sp.sin(w1 * t + phi12))
ee1 = get_act(rev1, rev2, act_len1)

# Calculating angles of rods
rod_angle1 = sp.atan2(rev2.y - rev1.y, rev2.x - rev1.x)

In [ ]:
# Precalculate all trajectories via substitution
rev1s = []
rev2s = []
ee1s = []
ra1s = []

for frame in ts:
    rev1s.append(rev1.subs(t, frame))
    rev2s.append(rev2.subs(t, frame))
    ee1s.append(ee1.subs(t, frame))
    ra1s.append(rod_angle1.subs(t, frame))
rev1s = np.array(rev1s, dtype=np.float64)
rev2s = np.array(rev2s, dtype=np.float64)
ee1s = np.array(ee1s, dtype=np.float64)
ra1s = np.array(ra1s, dtype=np.float64)

In [ ]:
# Precisely calculate end-effector trajectory
# For better velocity calculation
# Not necessary

ee1s_precise = []
precision_dt = 0.07
precision_steps = int((t_end - t_begin) / precision_dt)

cur_time = 0
for _ in range(precision_steps):
    ee1s_precise.append(ee1.subs(t, cur_time))
    if len(ee1s_precise) > 1 and ee1s_precise[-1] == ee1s_precise[0]:
        break
    cur_time += precision_dt

ee1s_precise = np.array(ee1s_precise, dtype=np.float64)

## Dynamics


### First task

Task: we have initial torques $\tau_1$ and $\tau_2$ for each wheel. 